In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount = 'True')

Mounted at /content/drive/


In [ ]:
!pip install transformers
!pip install rouge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 32.1 MB/s 
     |████████████████████████████████| 182 kB 82.2 MB/s 
     |████████████████████████████████| 7.6 MB 65.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install rouge.score nltk py7zr

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 65 kB 5.3 MB/s 
     |████████████████████████████████| 2.3 MB 54.6 MB/s 
     |████████████████████████████████| 139 kB 57.1 MB/s 
     |████████████████████████████████| 357 kB 82.7 MB/s 
     |████████████████████████████████| 94 kB 3.6 MB/s 
     |████████████████████████████████| 50 kB 4.2 MB/s 
     |████████████████████████████████| 378 kB 70.4 MB/s 
  Created wheel for rouge.score: filename=rouge_score-0.1.2-py3-none-any.whl size=24955 sha256=c9c7d3f05dfe00612051658cdb67907d460834da201fa711e25e815fdf96bbf6
  Stored in directory: /root/.cache/pip/wheels/24/55/6f/ebfc4cb176d1c9665da4e306e1705496206d08215c1acd9dde
Successfully built rouge.score


In [ ]:
!pip install transformers datasets 


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 10.0 MB/s 
     |████████████████████████████████| 212 kB 82.4 MB/s 
     |████████████████████████████████| 132 kB 76.0 MB/s 
     |████████████████████████████████| 127 kB 82.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
import numpy as np
import pandas as pd

import nltk
import transformers
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
nltk.download('punkt')

import re
import string
import csv
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import LSTM, Dense, Input, Embedding, Concatenate, TimeDistributed, Bidirectional, GRU
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from keras.utils.vis_utils import plot_model
from rouge import Rouge
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from datasets import load_dataset, load_metric

In [ ]:
dataset = load_dataset("csv", data_files={"train": "/content/drive/MyDrive/Statistical Software - MSDS Fall 2022/Data/train.csv", "validation": "/content/drive/MyDrive/Statistical Software - MSDS Fall 2022/Data/validation.csv", "test": "/content/drive/MyDrive/Statistical Software - MSDS Fall 2022/Data/test.csv"})

Extracting data files #0:   0%|          | 0/1 [00:00<?, ?obj/s]

Extracting data files #1:   0%|          | 0/1 [00:00<?, ?obj/s]

Extracting data files #2:   0%|          | 0/1 [00:00<?, ?obj/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-3b66c66cb98dc875/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
metric = load_metric('rouge')
model_checkpoints = 'facebook/bart-large-cnn'

<ipython-input-9-b0caef46fdcb>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('rouge')


In [ ]:
max_input = 512
max_target = 128
batch_size = 3
model_checkpoints = "facebook/bart-large-cnn"

In [ ]:
tokenizer = transformers.AutoTokenizer.from_pretrained(model_checkpoints)

Downloading:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
def preprocess_data(data_to_process):
  inputs = [text for text in data_to_process['article']]
  #tokenize text
  model_inputs = tokenizer(inputs,  max_length=max_input, padding='max_length', truncation=True)

  #tokenize highlights
  with tokenizer.as_target_tokenizer():
    targets = tokenizer(data_to_process['highlights'], max_length=max_target, padding='max_length', truncation=True)
    
  model_inputs['labels'] = targets['input_ids']
  #reuturns input_ids, attention_masks, labels
  return model_inputs

In [ ]:
tokenize_data = dataset.map(preprocess_data, batched = True, remove_columns=['id', 'article', 'highlights'])

  0%|          | 0/288 [00:00<?, ?ba/s]

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:3578: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


  0%|          | 0/14 [00:00<?, ?ba/s]

  0%|          | 0/12 [00:00<?, ?ba/s]

In [ ]:
tokenize_data

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 11490
    })
})

In [ ]:
#sample the data
train_sample = tokenize_data['train'].select(range(5000))
validation_sample = tokenize_data['validation'].select(range(1000))
test_sample = tokenize_data['test'].select(range(500))

In [ ]:
model = transformers.AutoModelForSeq2SeqLM.from_pretrained(model_checkpoints)

Downloading:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

In [ ]:
batch_size = 1
#collator to create batches. It preprocess data with the given tokenizer
collator = transformers.DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
#####################
# metrics
# compute rouge for evaluation 
#####################

def compute_rouge(pred):
  predictions, labels = pred
  #decode the predictions
  decode_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
  #decode labels
  decode_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

  #compute results
  res = metric.compute(predictions=decode_predictions, references=decode_labels, use_stemmer=True)
  #get %
  res = {key: value.mid.fmeasure * 100 for key, value in res.items()}

  pred_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
  res['gen_len'] = np.mean(pred_lens)

  return {k: round(v, 4) for k, v in res.items()}
     

In [ ]:
args = transformers.Seq2SeqTrainingArguments(
    'news-summ',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size= 1,
    gradient_accumulation_steps=2,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=3,
    predict_with_generate=True,
    eval_accumulation_steps=1,
    fp16=True
    )
#only CUDA available -> fp16=True
     

In [ ]:
tokenize_data

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 11490
    })
})

In [ ]:
train_sample

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 5000
})

In [ ]:
validation_sample

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 1000
})

In [ ]:
trainer = Seq2SeqTrainer(
    model, 
    args,
    train_dataset=train_sample,
    eval_dataset=validation_sample,
    data_collator=collator,
    tokenizer=tokenizer,
    compute_metrics=compute_rouge
)

Using cuda_amp half precision backend


In [ ]:
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 5000
  Num Epochs = 3
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 2
  Total optimization steps = 7500
  Number of trainable parameters = 406290432
You're using a BartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,0.765200,0.928794,43.233000,20.433800,29.351200,40.258600,105.616000
2,0.536600,0.980270,44.261100,20.910600,29.939200,41.373500,98.380000
3,0.342100,1.097859,44.409400,20.820900,30.201100,41.492200,95.305000


Saving model checkpoint to news-summ/checkpoint-500
Configuration saved in news-summ/checkpoint-500/config.json
Model weights saved in news-summ/checkpoint-500/pytorch_model.bin
tokenizer config file saved in news-summ/checkpoint-500/tokenizer_config.json
Special tokens file saved in news-summ/checkpoint-500/special_tokens_map.json
Saving model checkpoint to news-summ/checkpoint-1000
Configuration saved in news-summ/checkpoint-1000/config.json
Model weights saved in news-summ/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in news-summ/checkpoint-1000/tokenizer_config.json
Special tokens file saved in news-summ/checkpoint-1000/special_tokens_map.json
Saving model checkpoint to news-summ/checkpoint-1500
Configuration saved in news-summ/checkpoint-1500/config.json
Model weights saved in news-summ/checkpoint-1500/pytorch_model.bin
tokenizer config file saved in news-summ/checkpoint-1500/tokenizer_config.json
Special tokens file saved in news-summ/checkpoint-1500/special_toke

TrainOutput(global_step=7500, training_loss=0.5652245320638021, metrics={'train_runtime': 8710.3376, 'train_samples_per_second': 1.722, 'train_steps_per_second': 0.861, 'total_flos': 1.625328451584e+16, 'train_loss': 0.5652245320638021, 'epoch': 3.0})

In [ ]:
trainer.save_model("/content/drive/MyDrive/Statistical Software - MSDS Fall 2022/Data/my_fine_tuned_model")

Saving model checkpoint to /content/drive/MyDrive/Statistical Software - MSDS Fall 2022/Data/my_fine_tuned_model
Configuration saved in /content/drive/MyDrive/Statistical Software - MSDS Fall 2022/Data/my_fine_tuned_model/config.json
Model weights saved in /content/drive/MyDrive/Statistical Software - MSDS Fall 2022/Data/my_fine_tuned_model/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/Statistical Software - MSDS Fall 2022/Data/my_fine_tuned_model/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/Statistical Software - MSDS Fall 2022/Data/my_fine_tuned_model/special_tokens_map.json


In [ ]:
testing_article= """Ever noticed how plane seats appear to be getting smaller and smaller? With increasing numbers of people taking to the skies, some experts are questioning if having such packed out planes is putting passengers at risk. They say that the shrinking space on aeroplanes is not only uncomfortable - it's putting our health and safety in danger. More than squabbling over the arm rest, shrinking space on planes putting our health and safety in danger? This week, a U.S consumer advisory group set up by the Department of Transportation said at a public hearing that while the government is happy to set standards for animals flying on planes, it doesn't stipulate a minimum amount of space for humans. 'In a world where animals have more rights to space and food than humans,' said Charlie Leocha, consumer representative on the committee.Â 'It is time that the DOT and FAA take a stand for humane treatment of passengers.' But could crowding on planes lead to more serious issues than fighting for space in the overhead lockers, crashing elbows and seat back kicking? Tests conducted by the FAA use planes with a 31 inch pitch, a standard which on some airlines has decreased . Many economy seats on United Airlines have 30 inches of room, while some airlines offer as little as 28 inches . Cynthia Corbertt, a human factors researcher with the Federal Aviation Administration, that it conducts tests on how quickly passengers can leave a plane. But these tests are conducted using planes with 31 inches between each row of seats, a standard which on some airlines has decreased, reported the Detroit News. The distance between two seats from one point on a seat to the same point on the seat behind it is known as the pitch. While most airlines stick to a pitch of 31 inches or above, some fall below this. While United Airlines has 30 inches of space, Gulf Air economy seats have between 29 and 32 inches, Air Asia offers 29 inches and Spirit Airlines offers just 28 inches. British Airways has a seat pitch of 31 inches, while easyJet has 29 inches, Thomson's short haul seat pitch is 28 inches, and Virgin Atlantic's is 30-31."""

In [ ]:
model_inputs = tokenizer(testing_article,  max_length=max_input, padding='max_length', truncation=True)

In [ ]:
model_inputs

{'input_ids': [0, 27172, 5324, 141, 3286, 3202, 2082, 7, 28, 562, 2735, 8, 2735, 116, 590, 2284, 1530, 9, 82, 602, 7, 5, 12034, 6, 103, 2320, 32, 8026, 114, 519, 215, 6515, 66, 8449, 16, 2057, 3670, 23, 810, 4, 252, 224, 14, 5, 16921, 980, 15, 16482, 25934, 11372, 16, 45, 129, 9800, 111, 24, 18, 2057, 84, 474, 8, 1078, 11, 4854, 4, 901, 87, 9316, 37333, 81, 5, 3124, 1079, 6, 16921, 980, 15, 8449, 2057, 84, 474, 8, 1078, 11, 4854, 116, 152, 186, 6, 10, 121, 4, 104, 2267, 7640, 333, 278, 62, 30, 5, 641, 9, 6586, 26, 23, 10, 285, 1576, 14, 150, 5, 168, 16, 1372, 7, 278, 2820, 13, 3122, 4731, 15, 8449, 6, 24, 630, 75, 14808, 10246, 10, 3527, 1280, 9, 980, 13, 5868, 4, 128, 1121, 10, 232, 147, 3122, 33, 55, 659, 7, 980, 8, 689, 87, 5868, 1598, 26, 6127, 1063, 4306, 102, 6, 2267, 4915, 15, 5, 1540, 4, 2537, 50141, 108, 243, 16, 86, 14, 5, 30510, 8, 21419, 185, 10, 1413, 13, 26899, 1416, 9, 3670, 955, 125, 115, 2180, 154, 15, 8449, 483, 7, 55, 1473, 743, 87, 2190, 13, 980, 11, 5, 15258, 7014,

In [ ]:
raw_pred, _, _ = trainer.predict([model_inputs])

***** Running Prediction *****
  Num examples = 1
  Batch size = 1


In [ ]:
raw_pred

array([[    2,     0,     0,     0, 42909, 50141,   625, 20946, 50141,
        13839, 50141, 28357, 16921,   980,    15,  8449,    16,  2057,
           84,   474,     8,  1078,    11,  4854,   479, 50118,  3048,
         2267,  7640,   333,    26,    14,   150,     5,   168,    16,
         1372,     7,   278,  2820,    13,  3122,  4731,    15,  8449,
            6,    24,   630,    75, 14808, 10246,    10,  3527,  1280,
            9,   980,    13,  5868,   479, 50118,   565, 10092,  2964,
           30,     5, 21419,   304,  8449,    19,    10,  1105, 10468,
         2418,  3242,     6,    10,  2526,    61,    15,   103,  8537,
           34,  8065,   479, 50118, 20556,  6503,    34,   389,  4877,
            9,   980,     6,   150,  4602,  1754,   866,  3202,    33,
          227,  1132,     8,  2107,  4877,   479,     2,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,
      

In [ ]:
testing_summary = tokenizer.decode(raw_pred[0])

In [ ]:
testing_summary

"</s><s><s><s>Consumer\xa0advisory\xa0group\xa0say shrinking space on planes is putting our health and safety in danger.\nUS consumer advisory group said that while the government is happy to set standards for animals flying on planes, it doesn't stipulate a minimum amount of space for humans.\nTests conducted by the FAA use planes with a 31 inch seat pitch, a standard which on some airlines has decreased.\nUnited Airlines has 30 inches of space, while Gulf Air economy seats have between 29 and 32 inches.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>"

In [ ]:
testing_highlight = """Experts question if  packed out planes are putting passengers at risk .
U.S consumer advisory group says minimum space must be stipulated .
Safety tests conducted on planes with more leg room than airlines offer ."""

In [ ]:
rouge = Rouge()
rouge.get_scores(testing_summary, testing_highlight)

[{'rouge-1': {'r': 0.3333333333333333,
   'p': 0.15942028985507245,
   'f': 0.21568627013264136},
  'rouge-2': {'r': 0.12121212121212122,
   'p': 0.04819277108433735,
   'f': 0.06896551317033318},
  'rouge-l': {'r': 0.3333333333333333,
   'p': 0.15942028985507245,
   'f': 0.21568627013264136}}]

In [ ]:
#trying to test all 500 summaries

In [ ]:
test_sample

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 500
})

In [ ]:
test_sample[0]

{'input_ids': [0,
  27172,
  5324,
  141,
  3286,
  3202,
  2082,
  7,
  28,
  562,
  2735,
  8,
  2735,
  116,
  590,
  2284,
  1530,
  9,
  82,
  602,
  7,
  5,
  12034,
  6,
  103,
  2320,
  32,
  8026,
  114,
  519,
  215,
  6515,
  66,
  8449,
  16,
  2057,
  3670,
  23,
  810,
  4,
  252,
  224,
  14,
  5,
  16921,
  980,
  15,
  16482,
  25934,
  11372,
  16,
  45,
  129,
  9800,
  111,
  24,
  18,
  2057,
  84,
  474,
  8,
  1078,
  11,
  4854,
  4,
  901,
  87,
  9316,
  37333,
  81,
  5,
  3124,
  1079,
  6,
  16921,
  980,
  15,
  8449,
  2057,
  84,
  474,
  8,
  1078,
  11,
  4854,
  116,
  152,
  186,
  6,
  10,
  121,
  4,
  104,
  2267,
  7640,
  333,
  278,
  62,
  30,
  5,
  641,
  9,
  6586,
  26,
  23,
  10,
  285,
  1576,
  14,
  150,
  5,
  168,
  16,
  1372,
  7,
  278,
  2820,
  13,
  3122,
  4731,
  15,
  8449,
  6,
  24,
  630,
  75,
  14808,
  10246,
  10,
  3527,
  1280,
  9,
  980,
  13,
  5868,
  4,
  128,
  1121,
  10,
  232,
  147,
  3122,
  33,
  55,
  

In [ ]:
test_df = pd.read_csv('/content/drive/MyDrive/Statistical Software - MSDS Fall 2022/Data/test.csv')

In [ ]:
test_df.columns

Index(['id', 'article', 'highlights'], dtype='object')

In [ ]:
hyps = []
with open('/content/drive/MyDrive/Statistical Software - MSDS Fall 2022/Data/fine_tuned_bart-cnn_12_16_22.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['Article', 'Original Summary', 'Model Output'])
    for i in range(len(test_sample)):
      raw_pred, _, _ = trainer.predict([test_sample[i]])
      our_summ= tokenizer.decode(raw_pred[0])
      hyps.append(our_summ)
      writer.writerow([test_df.article[i], test_df.highlights[i], our_summ])

***** Running Prediction *****
  Num examples = 1
  Batch size = 1


***** Running Prediction *****
  Num examples = 1
  Batch size = 1


***** Running Prediction *****
  Num examples = 1
  Batch size = 1


***** Running Prediction *****
  Num examples = 1
  Batch size = 1


***** Running Prediction *****
  Num examples = 1
  Batch size = 1


...

***** Running Prediction *****
  Num examples = 1
  Batch size = 1


***** Running Prediction *****
  Num examples = 1
  Batch size = 1


***** Running Prediction *****
  Num examples = 1
  Batch size = 1


***** Running Prediction *****
  Num examples = 1
  Batch size = 1


***** Running Prediction *****
  Num examples = 1
  Batch size = 1


In [ ]:
fine_tuned_bart_cnn_12_16_22_results = pd.read_csv('/content/drive/MyDrive/Statistical Software - MSDS Fall 2022/Data/fine_tuned_bart-cnn_12_16_22.csv')

In [ ]:
rouge = Rouge()
rouge.get_scores(fine_tuned_bart_cnn_12_16_22_results['Original Summary'], fine_tuned_bart_cnn_12_16_22_results['Model Output'], avg=True, ignore_empty=True)

{'rouge-1': {'r': 0.3487579239190594,
  'p': 0.4650794065846994,
  'f': 0.39139958307704503},
 'rouge-2': {'r': 0.15196810364190633,
  'p': 0.20945564290952415,
  'f': 0.1717421914428125},
 'rouge-l': {'r': 0.32783443097487114,
  'p': 0.43724952134778666,
  'f': 0.3679594644708039}}